![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## This notebook is an example of how to stream AI annotated video with RTSP protocol. 
A video stream from local camera is processed by the person detection model. The media server is launched. The annotated video stream is then streamed out as RTSP stream to the media server.
The media server can serve both RTSP and and WebRTC streams to multiple clients.
Once this script is running, you can access the WebRTC stream at the following URL: http://localhost:8888/my-ai-stream/
You can access the RTSP stream at the following URL: rtsp://localhost:8554/my-ai-stream/

This example uses `degirum_tools.streams` streaming toolkit.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

The script can use either local camera or web camera connected to the machine, or a video file. The camera index or URL or video file path needs to be specified in the code below by assigning `video_source`.

You need to install MediaMTX and FFmpeg programs on your system, so they are available in the system PATH.
Please refer to https://github.com/bluenviron/mediamtx for MediaMTX installation instructions.
Please refer to https://ffmpeg.org/download.html for FFmpeg installation instructions.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences, model_zoo_url, model names for inference, and video source

In [1]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# people_det_model_name: name of the model for detecting people
# pose_det_model_name: name of the model for pose detection
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# url_path: path for the RTSP server to serve the video stream
hw_location = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5s_person_det--512x512_quant_n2x_orca1_1"
video_source = 0 # local camera
url_path = "/my-ai-stream"

#### The rest of the cells below should run without any modifications

In [4]:
import degirum as dg, degirum_tools, time
from degirum_tools import streams as dgstreams

# load model
model = dg.load_model(
    model_name,
    hw_location,
    model_zoo_url,
    degirum_tools.get_token(),
    overlay_show_probabilities=True,
    overlay_line_width=1,
)

# create gizmos

# video source gizmo
cam_source = dgstreams.VideoSourceGizmo(video_source)

# detection gizmo
detector = dgstreams.AiSimpleGizmo(model)

# video streamer gizmo
streamer = dgstreams.VideoStreamerGizmo(f"rtsp://localhost:8554{url_path}", show_ai_overlay=True)

# local display gizmo (just for debugging)
display = dgstreams.VideoDisplayGizmo(show_ai_overlay=True)

# start media server to serve RTSP streams
with degirum_tools.MediaServer():
    # connect gizmos into pipeline and start composition
    dgstreams.Composition(cam_source >> detector >> streamer, detector >> display).start()

#
# You can access the WebRTC stream at the following URL: http://localhost:8888/my-ai-stream/
# You can access the RTSP stream at the following URL: rtsp://localhost:8554/my-ai-stream/
#